<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-set-up-functions" data-toc-modified-id="Imports-and-set-up-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and set-up functions</a></span></li><li><span><a href="#Define-the-PDE-for-Sub-Surface-Temperature-(SST)" data-toc-modified-id="Define-the-PDE-for-Sub-Surface-Temperature-(SST)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define the PDE for Sub-Surface Temperature (SST)</a></span></li><li><span><a href="#Simulate-SST-data" data-toc-modified-id="Simulate-SST-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simulate SST data</a></span><ul class="toc-item"><li><span><a href="#Method-1:-Harmonic-approximation-of-motion-vector-w" data-toc-modified-id="Method-1:-Harmonic-approximation-of-motion-vector-w-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Method 1: Harmonic approximation of motion vector w</a></span></li><li><span><a href="#Generate-simulated-SST-data" data-toc-modified-id="Generate-simulated-SST-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Generate simulated SST data</a></span></li></ul></li><li><span><a href="#Predict-future-SST-states" data-toc-modified-id="Predict-future-SST-states-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Predict future SST states</a></span><ul class="toc-item"><li><span><a href="#CDNN" data-toc-modified-id="CDNN-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>CDNN</a></span></li></ul></li></ul></div>

# Imports and set-up functions

In [26]:
import os
import math
import numpy as np

In [4]:
def dataDir(x):
    cwd = os.getcwd()
    
    return cwd + x 

# Define the PDE for Sub-Surface Temperature (SST)

From (Bézenac, Pajot, and Gallinari, 2019), Equation 3 (p. 2) gives the linear, elliptic PDE for advection-diffusion: 

$\begin{equation}
\frac{\partial I}{\partial t} + (\omega . \nabla)I = D\nabla^2 I
\tag{1}
\label{eq1}
\end{equation}$

where $\omega \in \mathbb{R^2}$ is the motion vector $\frac{\Delta x}{\Delta t}$, $D$ is the diffusion coefficient, $\nabla$ denotes the gradient operator, and $\nabla^2$ denotes the Laplacian operator.


**Theorem 1** states that, for any initial condition $I_0 \in L^1(\mathbb{R}^2)$ with $I_0(\pm \infty) = 0$, there exists a unique global solution $I(x, t)$ to $\eqref{eq1}$:

$\begin{equation}
I(x, t) = \int_{\mathbb{R}^2} k(x - w, y) I_0(y)dy
\tag{2}
\label{eq2}
\end{equation}$

Where $k(u, v) = \frac{1}{4\pi Dt}e^{-\frac{1}{4Dt}||u-v||^2}$ is a radial basis function kernel, or alternatively, a 2-dimensional Gaussian probability density with mean $u$ and variance $2Dt$.

All this means that if the initial condition $I_0$, the diffusion coefficient $D$, and the motion vector $\omega$ were known, future temperatures could be predicted from past ones. Unfortunately in practice, none of these quantities are known; they must be estimated from the data. Below, we simulate test data and use it to estimate the motion vector $\omega$ using a Convolution-Deconvolution Neural Network (CDNN). 

# Simulate SST data 

## Method 1: Harmonic approximation of motion vector w

Let us define an approximation function for the motion vector $\omega(x, t)$, for $x \in \mathbb{R^2}$: 

$\begin{equation}
\hat{\omega}(t, x) = \beta_0(x) + \beta_1(x)\sin(\frac{2\pi t}{24}) + \beta_2(x)\sin(\frac{2\pi t}{12}) + \epsilon
\tag{3}
\label{eq3}
\end{equation}$

where $\epsilon$ is some multivariate Gaussian noise with mean $\mu \in \mathbb{R^2}$ and covariance matrix $\Sigma \in \mathbb{R^{2x2}}$. The scalar values $24$ and $12$ represent the period of the harmonic, for example if $t$ is expressed in hours. 

## Generate simulated SST data

In [32]:
# Implement a function for the beta(x) terms

def betas(x, const=3):
    """A simple function to evaluate the beta(x) terms in equation 3.
    
       :param x: a 2x1 vector of positions on a grid
       :type x: numpy.ndarray
       
       :return betas: a 3x1 vector of wind parameters
       :type betas: numpy.ndarray
    """
    
    # Verify dimensions
    assert(x.shape == (2, 1))
    
    betas = np.array([[const], [const*x[0]], [(const**2)*x[1]]])
    
    return betas

In [33]:
# Implement the function that approximates the motion/wind vector (equation 3)

def wind(x, t):
    """A harmonic function with two different periods that approximates the 
       wind vector, as given by equation 3. The function depends on 
       position x, time t, and beta parameters. 
       
       :param x: a 2x1 position vector (i, j) 
       :type x: numpy.ndarray
       
       :param t: a scalar time
       :type t: float
       
       :return omega: a 2x1 vector of wind intensities
       :type omega: numpy.ndarray
    """
    
    # Verify dimensions
    assert(x.shape == (2, 1))
    
    harm1 = math.sin((2*math.pi*t)/24)
    harm2 = math.sin((2*math.pi*t)/12)
    
    # Get beta parameters
    Bs = betas(x)
    assert(Bs.shape == (3, 1))
    
    # Compute wind
    w = Bs[0] + Bs[1]*harm1 + Bs[2]*harm2
    
    return w

# Predict future SST states

## CDNN